In [11]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")
llm.invoke("The first man on the moon was ...")

/var/folders/_9/nxb2f4f13px9by3vcfvrckp40000gn/T/ipykernel_47769/194194787.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


'Neil Armstrong! He stepped out of the lunar module Eagle and onto the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first person to set foot on the moon.'

In [10]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="llama3", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)
llm.invoke("The first man on the moon was ...")

/var/folders/_9/nxb2f4f13px9by3vcfvrckp40000gn/T/ipykernel_20467/2140520511.py:4: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(


Neil Armstrong! On July 20, 1969, Neil Armstrong and Edwin "Buzz" Aldrin became the first humans to set foot on the Moon as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he stepped off the lunar module Eagle onto the Moon's surface.

'Neil Armstrong! On July 20, 1969, Neil Armstrong and Edwin "Buzz" Aldrin became the first humans to set foot on the Moon as part of the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he stepped off the lunar module Eagle onto the Moon\'s surface.'

In [16]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [18]:
from sentence_transformers import SentenceTransformer
from typing import List
    
class MyEmbeddings:
    def __init__(self, model):
        self.model = SentenceTransformer(model, trust_remote_code=True)
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self.model.encode(t).tolist() for t in texts]
            
    def embed_query(self, query: str) -> List[float]:
        return self.model.encode([query]).tolist()

/Users/aryanbarik/Code/nstarx/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
from langchain_ollama import OllamaEmbeddings
embed = OllamaEmbeddings(
    model="llama3"
)

In [24]:
input_text = "The meaning of life is 42"
vector = embed.embed_query(input_text)
print(vector[:3])

[0.014715631, -0.0037208814, -0.00076697685]


In [23]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# https://stackoverflow.com/questions/77217193/langchain-how-to-use-a-custom-embedding-model-locally
vectorstore = Chroma.from_documents(documents=splits, embedding=OllamaEmbeddings(model="llama3"))


# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

InvalidDimensionException: Embedding dimension 4096 does not match collection dimensionality 1024

In [28]:
rag_chain.invoke("What is Task Decomposition?")

ValueError: Expected embeddings to be a list of floats or ints, a list of lists, a numpy array, or a list of numpy arrays, got [[[0.0034616945777088404, 0.0015415868256241083, 0.018468406051397324, 0.02181551605463028, -0.03861014544963837, 0.06722038984298706, -0.04830658808350563, -0.03027186170220375, -0.01336503867059946, 0.001540154218673706, 0.025010429322719574, -0.018276656046509743, -0.012959412299096584, 0.008468745276331902, -0.025917181745171547, 0.01732330396771431, 0.010883983224630356, -0.052047669887542725, 0.03249093517661095, -0.0048014381900429726, 0.021406253799796104, -0.008949948474764824, -0.010673483833670616, -0.03578928858041763, 0.011305581778287888, 0.0038023816887289286, 0.0023997249081730843, 0.020839978009462357, 0.04613739997148514, 0.014657679945230484, -0.021194541826844215, 0.023243065923452377, -0.029993586242198944, -0.022755635902285576, -0.015597694553434849, -0.03856169059872627, -0.001305036130361259, -0.010740541853010654, 0.021492280066013336,

# Local Llama3 Rag with Ollama Setup

In [22]:
import fitz  # PyMuPDF

def pdf_to_text(pdf_path):
    # Open the PDF file
    document = fitz.open(pdf_path)
    text = ""

    # Iterate through each page
    for page_num in range(len(document)):
        page = document.load_page(page_num)  # Load the page
        text += page.get_text()  # Extract text from the page

    return text

# Example usage
pdf_path = "/Users/aryanbarik/Downloads/nstarx-corpus/February2025TalbotsCatalog.pdf"  # Replace with your PDF file path
text = pdf_to_text(pdf_path)
print(text)

FEBRUARY 2025
look good
ok good
feel good
look good
feel good
Longer, brighter days are just ahead,  
and our new collection is here and ready  
to make this your feel-good February. 
Gorgeous colors. Tweeds and cozy textures.  
Great-fitting pants to freshen up your 
rotation. Beautiful matching sets with endless 
versatility. It’s pulled-together perfection for 
all the sunshine around the bend.
TALBOTS © 2025
Front Cover:
A. Floral Medallion Ruffle Top
Band neck. Long sleeve. At hip. M251/2"; 
P241/2"; W281/2"; WP271/2". Fully lined. 
Polyester. Machine wash. Mulberry wine/
rose sorbet. A01511
M XS-XL 	 P P-XL 	
$99.50
W X-3X 	
WP X-3X 	
$109
INDIGO
B. Ribbed Midi Skirt  Pull on. M301/2"; 
P281/2"; W293/4"; WP273/4". Viscose. 
Machine wash. Mulberry wine. B03511 
M XS-XL 	 P P-XL 	
$119
W X-3X 	
WP X-3X 	
$139
INDIGO
A. Slim Johnny Collar Pullover V-neck. 
Long sleeve. At hip. Viscose/LYCRA®. 
Machine wash. Mulberry wine. A03511  
M XS-XL	
P P-XL 	
$99.50
W X-3X 	
WP X-3X 	
$109
C. 

In [23]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Initialize the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create a FAISS index
dimension = 384  # Dimension of the embeddings
index = faiss.IndexFlatL2(dimension)

# Example documents
documents = [text]

# Generate embeddings for the documents
document_embeddings = embedding_model.encode(documents)

# Add embeddings to the FAISS index
index.add(np.array(document_embeddings))

/Users/aryanbarik/Code/nstarx/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
import ollama

def retrieve_relevant_documents(query, top_k=2):
    # Generate query embedding
    query_embedding = embedding_model.encode([query])
    
    # Search the FAISS index for the most relevant documents
    distances, indices = index.search(np.array(query_embedding), top_k)
    
    # Retrieve the relevant documents
    relevant_documents = [documents[i] for i in indices[0]]
    return relevant_documents

def generate_response(query):
    # Retrieve relevant documents
    relevant_docs = retrieve_relevant_documents(query)
    
    # Combine the query and relevant documents into a prompt
    context = "\n".join(relevant_docs)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    
    # Generate response using Llama3 through Ollama
    response = ollama.generate(model='llama3', prompt=prompt)
    
    return response['response']

# Example query
query = "Give a summary of the document."
response = generate_response(query)
print(response)

The document appears to be a Talbots clothing catalog for February 2025. It features a variety of clothing items, including tops, bottoms, dresses, outerwear, and accessories, in various colors and patterns. The catalog includes descriptions of each item, prices, and sizes available. There are also sections on new arrivals, customer service information, and ways to shop or contact Talbots.


In [25]:
queries = ['Give a summary of the document.', 
'When was the catalog published?' ,
'What is the description of the Slim Johnny Collar Pullover?' ,
'What are the prices for the Slim Johnny Collar Pullover?',
'What is the description of the Collarless Tweed Jacket?',
'What are the prices for the Collarless Tweed Jacket?',
'What are the prices for the Ribbed Midi Skirt?',
'What is the price for the Faceted Stones Necklace?',
'What are the prices of the Floral Medallion Ruffle Top?',
'What are the prices of the Jasmine Stripe Ribbed Crewneck Tee?',
'What is the description of the Jasmine Stripe Ribbed Crewneck Tee?']


In [26]:
responses = []

for query in queries:
    responses.append(generate_response(query))
    
responses

['The document appears to be a Talbots clothing catalog for February 2025. It features a variety of clothing items, including tops, bottoms, dresses, and outerwear, in various styles, colors, and sizes. The catalog includes descriptions of each item, prices, and ordering information. It also includes sections on new arrivals, customer service, and social media links.',
 'According to the information provided in the catalog, it was published in FEBRUARY 2025.',
 'There is no mention of a "Slim Johnny Collar Pullover" in this catalog. The closest match would be the "Ribbed Johnny Collar Pullover" mentioned on page 76, which has the description: "Long sleeve. At hip. Cotton/polyester. Machine wash."',
 "According to the catalog, the prices for the Ribbed Johnny Collar Pullover (A76511) are:\n\n* Men's XS-XL: $89.50\n* Women's X-3X: $99.50\n\nPlease note that these prices only apply to this specific style and size. Prices may vary depending on other factors, such as location or promotions.